<a href="https://colab.research.google.com/github/SushmitaSingh96/Sound_Recognition_-_Cancellation/blob/main/Sound_Recognition_And_Cancellation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#pip install -r requirements.txt

#YAMNET to recognise the different sounds present in an audio file

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv

import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io import wavfile

In [15]:
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

In [16]:
# Find the name of the class with the top score when mean-aggregated across frames.
def class_names_from_csv(class_map_csv_text):
  """Returns list of class names corresponding to score vector."""
  class_names = []
  with tf.io.gfile.GFile(class_map_csv_text) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      class_names.append(row['display_name'])

  return class_names

class_map_path = model.class_map_path().numpy()
class_names = class_names_from_csv(class_map_path)

In [18]:
#convert audio to wav
from pydub import AudioSegment

# Load the M4A file
audio = AudioSegment.from_file("Speaking_and_appluse.m4a", format="m4a")

# Set the sample rate to 16kHz
audio = audio.set_frame_rate(16000)

# Convert to mono if not already
if audio.channels > 1:
    audio = audio.set_channels(1)

# Export as WAV file
audio.export("Speaking_and_appluse.wav", format="wav")

<_io.BufferedRandom name='Speaking_and_appluse.wav'>

In [19]:
#sample the audio waveform to 16kHz
def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required."""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)
  return desired_sample_rate, waveform

In [20]:
# wav_file_name = 'CoughAndClap.wav'
wav_file_name = 'Speaking_and_appluse.wav'
sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)

# Show some basic information about the audio.
duration = len(wav_data)/sample_rate
print(f'Sample rate: {sample_rate} Hz')
print(f'Total duration: {duration:.2f}s')
print(f'Size of the input: {len(wav_data)}')

# Listening to the wav file.
Audio(wav_data, rate=sample_rate)

Sample rate: 16000 Hz
Total duration: 3.46s
Size of the input: 55357


In [21]:
waveform = wav_data / tf.int16.max

In [22]:
# Run the model, check the output.
scores, embeddings, spectrogram = model(waveform)

In [23]:
scores_np = scores.numpy()
spectrogram_np = spectrogram.numpy()
infered_class = class_names[scores_np.mean(axis=0).argmax()]
print(f'The main sound is: {infered_class}')
print(f'Sample rate: {sample_rate} Hz')
print(f'Total duration: {duration:.2f}s')
print(f'Size of the input: {len(wav_data)}')

The main sound is: Speech
Sample rate: 16000 Hz
Total duration: 3.46s
Size of the input: 55357


#DeepFilterNet to supress noise

In [29]:
#deepfilternet only supports wav files with a sampling rate of 48kHz are supported. Convert the wav file to 48kHz sampling rate
from pydub import AudioSegment

# Load the input file
audio = AudioSegment.from_file("Speaking_and_appluse.wav")

# Set the desired sampling rate (48kHz)
target_sr = 48000

# Resample the audio to the target sampling rate and export it
audio.export("Speaking_and_appluse_48kHz.wav", format="wav", parameters=["-ar", str(target_sr)])


<_io.BufferedRandom name='Speaking_and_appluse_48kHz.wav'>

In [30]:
# Specify an output directory with --output-dir [OUTPUT_DIR]
!deepFilter Speaking_and_appluse_48kHz.wav

/usr/local/lib/python3.10/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2024-04-09 12:56:46 | INFO     | DF | Running on torch 2.2.1+cu121
2024-04-09 12:56:46 | INFO     | DF | Running on host f909a058a3e0
fatal: not a git repository (or any of the parent directories): .git
2024-04-09 12:56:46 | INFO     | DF | Loading model settings of DeepFilterNet3
2024-04-09 12:56:46 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2024-04-09 12:56:46 | INFO     | DF | Initializing model `deepfilternet3`
2024-04-09 12:56:46 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2024-04-09 12:56:46 | INFO     | DF | Running on device cpu
2024-04-09 12:56:46 | INFO     | DF | Model loaded
2024-04-09 12:56:47 | INFO     | DF | Enh